In [6]:
import requests

def get_references_from_doi(doi):
    """Fetches references from Crossref given a DOI."""
    url = f"https://api.crossref.org/works/{doi}"
    headers = {"User-Agent": "your_email@example.com (Python script for academic research)"}

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code}")
        return []

    data = response.json()

    # Navigate to the reference section if available
    message = data.get("message", {})
    references = message.get("reference", [])

    formatted_refs = []
    for ref in references:
        formatted = {
            "key": ref.get("key"),
            "DOI": ref.get("DOI"),
            "article-title": ref.get("article-title"),
            "author": ref.get("author"),
            "year": ref.get("year"),
            "journal-title": ref.get("journal-title"),
        }
        formatted_refs.append(formatted)

    return formatted_refs

# Example usage
if __name__ == "__main__":
    example_doi = "10.1038/s41586-020-2649-2"  # Change this to any valid DOI
    example_doi = "10.1162/neco.1992.4.1.131"  # Change this to any valid DOI
    example_doi = "10.48550/arXiv.1905.01664"
    #     example_doi_with_refs = "10.1162/neco.1992.4.1.131" # A Nature paper Attention is all you need

    refs = get_references_from_doi(example_doi)

    print(f"Found {len(refs)} references:")
    for i, ref in enumerate(refs, 1):
        print(f"{i}. {ref.get('author', 'N/A')} ({ref.get('year', 'N.D.')}): {ref.get('article-title', 'No title')}")


Error: Received status code 404
Found 0 references:


In [2]:
import requests
import json

def get_crossref_references(doi):
    """
    Fetches the references for a given DOI from the Crossref API.

    Args:
        doi (str): The Digital Object Identifier (DOI) of the work.

    Returns:
        dict or None: A dictionary containing the 'references' array if found,
                      otherwise None. Prints error messages on failure.
    """
    base_url = "https://api.crossref.org/works/"
    url = f"{base_url}{doi}"

    # It's good practice to provide a polite user-agent for API calls.
    # Replace 'YourAppName/1.0 (mailto:your_email@example.com)' with your actual info.
    headers = {
        "User-Agent": "MyResearchTool/1.0 (mailto:your.email@example.com)"
    }

    print(f"Attempting to fetch references for DOI: {doi}")
    print(f"API URL: {url}")

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)

        data = response.json()

        # Crossref API response structure: data -> message -> references
        if 'message' in data and 'references' in data['message']:
            references = data['message']['references']
            print(f"Found {len(references)} references for DOI: {doi}")
            return references
        elif 'message' in data and 'references' not in data['message']:
            print(f"No 'references' field found for DOI: {doi}. "
                  "This might mean the publisher did not deposit reference metadata.")
            return None
        else:
            print(f"Unexpected API response structure for DOI: {doi}")
            return None

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - Status Code: {response.status_code}")
        if response.status_code == 404:
            print(f"DOI '{doi}' not found in Crossref.")
        return None
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
        return None
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"An unexpected request error occurred: {req_err}")
        return None
    except json.JSONDecodeError as json_err:
        print(f"Error decoding JSON response: {json_err}")
        print(f"Response content: {response.text}")
        return None


In [3]:

# --- Example Usage ---
if __name__ == "__main__":
    # Example DOI for a paper that is likely to have references
    example_doi_with_refs = "10.1162/neco.1992.4.1.131" # A Nature paper Attention is all you need
    #example_doi_with_refs = "10.1038/s41586-023-06617-6" # A Nature paper
    #example_doi_no_refs = "10.5555/12345678" # A likely non-existent DOI or one without refs

    print("\n--- Testing DOI with References ---")
    references_data = get_crossref_references(example_doi_with_refs)

    if references_data:
        print("\nFirst 5 references (if available):")
        for i, ref in enumerate(references_data[:5]):
            print(f"  Reference {i+1}:")
            # Print common fields, handling cases where they might be missing
            print(f"    DOI: {ref.get('DOI', 'N/A')}")
            print(f"    Unstructured: {ref.get('unstructured', 'N/A')}")
            print(f"    Author: {ref.get('author', 'N/A')}")
            print(f"    Year: {ref.get('year', 'N/A')}")
            print(f"    Article Title: {ref.get('article-title', 'N/A')}")
            print(f"    Journal Title: {ref.get('journal-title', 'N/A')}")
            print("-" * 20)
    else:
        print(f"Could not retrieve references for {example_doi_with_refs}")

    print("\n--- Testing a DOI that might not exist or have references ---")
    no_refs_data = get_crossref_references(example_doi_no_refs)
    if no_refs_data is None:
        print(f"As expected, no references or error for {example_doi_no_refs}")

    print("\n--- Testing another DOI ---")
    another_doi = "10.1007/s11042-022-13271-x" # Another example
    another_references = get_crossref_references(another_doi)
    if another_references:
        print(f"Found {len(another_references)} references for {another_doi}")
    else:
        print(f"Could not retrieve references for {another_doi}")


--- Testing DOI with References ---
Attempting to fetch references for DOI: 10.1162/neco.1992.4.1.131
API URL: https://api.crossref.org/works/10.1162/neco.1992.4.1.131
No 'references' field found for DOI: 10.1162/neco.1992.4.1.131. This might mean the publisher did not deposit reference metadata.
Could not retrieve references for 10.1162/neco.1992.4.1.131

--- Testing a DOI that might not exist or have references ---
Attempting to fetch references for DOI: 10.5555/12345678
API URL: https://api.crossref.org/works/10.5555/12345678
No 'references' field found for DOI: 10.5555/12345678. This might mean the publisher did not deposit reference metadata.
As expected, no references or error for 10.5555/12345678

--- Testing another DOI ---
Attempting to fetch references for DOI: 10.1007/s11042-022-13271-x
API URL: https://api.crossref.org/works/10.1007/s11042-022-13271-x
HTTP error occurred: 404 Client Error: Not Found for url: https://api.crossref.org/works/10.1007/s11042-022-13271-x - Statu